In [1]:
import os
import zipfile
import shutil
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt


In [4]:
# Define the source and destination paths
source_dir = '/content/drive/MyDrive/ColabNotebooks/NewDataset/NITRC-multi-file-downloads'
destination_dir = '/content/drive/MyDrive/ColabNotebooks/NewDataset/processed'

# List of file patterns to extract
file_patterns = ['_flair_LPS_rSRI.nii.gz', '_t1_LPS_rSRI.nii.gz', '_t1ce_LPS_rSRI.nii.gz', '_t2_LPS_rSRI.nii.gz']

# Function to extract files from zip
def extract_files_from_zip(zip_path, dest_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if any(pattern in file for pattern in file_patterns):
                zip_ref.extract(file, dest_path)
                # Move the file to the correct sub-directory structure
                file_path = os.path.join(dest_path, file)
                dest_file_path = os.path.join(dest_path, os.path.basename(file))
                shutil.move(file_path, dest_file_path)
                # Clean up empty directories
                try:
                    os.removedirs(os.path.dirname(file_path))
                except OSError:
                    pass

# Function to traverse directories and extract files
def traverse_and_extract(source, destination):
    for root, dirs, files in os.walk(source):
        for file in files:
            if file.endswith('.zip'):
                zip_path = os.path.join(root, file)
                rel_path = os.path.relpath(root, source)
                dest_path = os.path.join(destination, rel_path)
                os.makedirs(dest_path, exist_ok=True)
                extract_files_from_zip(zip_path, dest_path)
            elif any(pattern in file for pattern in file_patterns):
                file_path = os.path.join(root, file)
                rel_path = os.path.relpath(root, source)
                dest_path = os.path.join(destination, rel_path)
                os.makedirs(dest_path, exist_ok=True)
                shutil.copy(file_path, dest_path)

# Run the extraction process
traverse_and_extract(source_dir, destination_dir)

In [ ]:
merged_data=pd.read_csv("/content/clinical_2014-01-16.csv")

In [ ]:
merged_data.head()

In [ ]:
merged_data.columns

In [ ]:
# Function to rename files based on disease label
def rename_files_with_disease_label(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.nii.gz'):
                # Extract the subject ID from the file name
                subject_id = file.split('_')[0]
                if subject_id in merged_data['Sample'].values:
                    disease = merged_data[merged_data['Sample'] == subject_id]['Disease'].values[0]
                else:
                    disease = 'unknown'  # Default label if subject ID is not found

                # Create new filename with disease label
                parts = file.split('_')
                parts.insert(1, disease)
                new_filename = '_'.join(parts)

                # Construct full paths for old and new filenames
                old_file_path = os.path.join(root, file)
                new_file_path = os.path.join(root, new_filename)

                # Rename the file
                os.rename(old_file_path, new_file_path)
                print(f"Renamed '{file}' to '{new_filename}'")

# Run the renaming process
rename_files_with_disease_label(destination_dir)

In [ ]:
filtered_files = [f for f in os.listdir(source_dir) if f.startswith('HF1551_ASTROCYTOMA') and f.endswith('.nii.gz')]
# Display images
for file in filtered_files:
    file_path = os.path.join(data_dir, file)
    img = nib.load(file_path)
    img_data = img.get_fdata()

    # Display the middle slice of each dimension
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img_data[:, :, img_data.shape[2] // 2], cmap='gray')
    axes[0].set_title('Axial')
    axes[1].imshow(img_data[:, img_data.shape[1] // 2, :], cmap='gray')
    axes[1].set_title('Coronal')
    axes[2].imshow(img_data[img_data.shape[0] // 2, :, :], cmap='gray')
    axes[2].set_title('Sagittal')
    plt.suptitle(file)
    plt.show()

In [ ]:
import os
import nibabel as nib
import numpy as np
import subprocess
import cv2
import matplotlib.pyplot as plt

# Install HD-BET dependencies and clone the repository
!git clone https://github.com/MIC-DKFZ/HD-BET.git
%cd HD-BET
!pip install -e .

# Function to perform skull stripping using HD-BET
def skull_strip_hd_bet(input_file, output_file):
    command = f"hd-bet -i {input_file} -o {output_file} -mode fast"
    subprocess.run(command, shell=True, check=True)

# Function to apply CLAHE
def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    # Ensure the image is in 8-bit format
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
        image = image.astype(np.uint8)

    # Create a CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    # Apply CLAHE to the grayscale image
    equalized_image = clahe.apply(image)
    return equalized_image

# Function to process a 3D volume along a specific axis
def process_volume_along_axis(volume, axis, process_func):
    processed_volume = np.zeros_like(volume)
    if axis == 0:  # Sagittal
        for i in range(volume.shape[0]):
            processed_volume[i, :, :] = process_func(volume[i, :, :])
    elif axis == 1:  # Coronal
        for i in range(volume.shape[1]):
            processed_volume[:, i, :] = process_func(volume[:, i, :])
    elif axis == 2:  # Axial
        for i in range(volume.shape[2]):
            processed_volume[:, :, i] = process_func(volume[:, :, i])
    return processed_volume

# Function to display 3D MRI volume slices
def display_volume_slices(volume, title):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    slice_0 = volume[volume.shape[0] // 2, :, :]
    slice_1 = volume[:, volume.shape[1] // 2, :]
    slice_2 = volume[:, :, volume.shape[2] // 2]

    axes[0].imshow(slice_0.T, cmap='gray', origin='lower')
    axes[0].set_title(f'{title} - Sagittal Slice')

    axes[1].imshow(slice_1.T, cmap='gray', origin='lower')
    axes[1].set_title(f'{title} - Coronal Slice')

    axes[2].imshow(slice_2.T, cmap='gray', origin='lower')
    axes[2].set_title(f'{title} - Axial Slice')

    plt.show()

# Path to the directory containing NIfTI files
data_dir = '/content/drive/MyDrive/ColabNotebooks/NewDataset/extracted-NITRC-multi-file-downloads'
filtered_files = [f for f in os.listdir(data_dir) if f.endswith('.nii.gz')]

for file_name in filtered_files:
    # Load the NIfTI file
    file_path = os.path.join(data_dir, file_name)
    nifti_image = nib.load(file_path)
    image_data = nifti_image.get_fdata()

    # Perform skull stripping using HD-BET
    stripped_output_file = os.path.join(data_dir, f"stripped_{file_name}")
    skull_strip_hd_bet(file_path, stripped_output_file)

    # Load the skull-stripped NIfTI file
    stripped_nifti_image = nib.load(stripped_output_file)
    stripped_image_data = stripped_nifti_image.get_fdata()

    # Display the results
    display_volume_slices(image_data, 'Original')
    display_volume_slices(stripped_image_data, 'Skull Stripped (HD-BET)')


    # Save the processed volumes as new NIfTI files
    clahe_img = nib.Nifti1Image(clahe_volume, stripped_nifti_image.affine)

    clahe_file_path = os.path.join(data_dir, f"clahe_{file_name}")

    nib.save(clahe_img, clahe_file_path)

    print(f"Processed files saved as {stripped_output_file} and {clahe_file_path}")
